In [1]:
import pandas as pd
import numpy as np
from scipy.stats import binned_statistic_2d
%pylab inline
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
import warnings
from itertools import product

Populating the interactive namespace from numpy and matplotlib


In [2]:
d = ["yellow_tripdata_2016-06.csv", "yellow_tripdata_2016-05.csv", "yellow_tripdata_2016-04.csv", "yellow_tripdata_2016-03.csv", "yellow_tripdata_2016-02.csv", "yellow_tripdata_2016-01.csv"
     , "yellow_tripdata_2015-12.csv", "yellow_tripdata_2015-11.csv", "yellow_tripdata_2015-10.csv", "yellow_tripdata_2015-09.csv", "yellow_tripdata_2015-08.csv", "yellow_tripdata_2015-07.csv"
    , "yellow_tripdata_2015-06.csv"]

In [3]:
k= pd.date_range(start='2015-06-01 00:00:00', end= '2016-06-30 23:00:00', freq='H')
reg= pd.read_csv("regions.csv", delimiter=';')
t = pd.DataFrame(index=range(len(k)),columns=range(len(reg.region)))
t.columns= reg.region
t.index= k
t.loc[:,:] = 0

In [4]:
x_array = pd.unique(reg.west)
x_array.sort()
y_array = pd.unique(reg.south)
y_array.sort()
x_array = np.append(x_array, reg.east[-1:].values)
y_array = np.append(y_array, reg.north[-1:].values)

In [5]:
da= []
def get(dat):
    data = pd.read_csv(str(dat))
    data = data[data.tpep_pickup_datetime != data.tpep_dropoff_datetime]
    data = data[data.passenger_count != 0]
    data = data[data.trip_distance != 0]
    data = data[data.pickup_longitude >= -74.25559]
    data = data[data.pickup_longitude <= -73.70001]
    data = data[data.pickup_latitude >= 40.49612]
    data = data[data.pickup_latitude <= 40.91553]
    data.tpep_pickup_datetime = pd.to_datetime(data.tpep_pickup_datetime)
    data.tpep_dropoff_datetime = pd.to_datetime(data.tpep_dropoff_datetime)
    data.tpep_pickup_datetime = data.tpep_pickup_datetime.map(lambda x: x.replace(minute=0, second=0))
    data.tpep_dropoff_datetime = data.tpep_dropoff_datetime.map(lambda x: x.replace(minute=0, second=0))
    a=stats.binned_statistic_2d(data.pickup_longitude, data.pickup_latitude, None, statistic = "count", bins = [x_array, y_array], expand_binnumbers=True)
    data['reg'] = (a[3][0] - 1) * 50 + (a[3][1]) #-1
    for i in range(len(data)):
        t[data.reg.iloc[i]][data.tpep_pickup_datetime.iloc[i]] += 1
       
    
    
    #da.append(data.tpep_pickup_datetime)
    return da

In [6]:
%%time
for i in range(len(d)):
    get(d[i])
    print(d[i])

yellow_tripdata_2016-06.csv
yellow_tripdata_2016-05.csv
yellow_tripdata_2016-04.csv
yellow_tripdata_2016-03.csv
yellow_tripdata_2016-02.csv
yellow_tripdata_2016-01.csv
yellow_tripdata_2015-12.csv
yellow_tripdata_2015-11.csv
yellow_tripdata_2015-10.csv
yellow_tripdata_2015-09.csv
yellow_tripdata_2015-08.csv
yellow_tripdata_2015-07.csv
yellow_tripdata_2015-06.csv
Wall time: 5h 52min 49s


In [7]:
data_mean = pd.DataFrame(t.loc['2016-05-01 00:00:00':'2016-06-01 00:00:00',:].mean(axis=0))
data_mean = data_mean[data_mean >= 5]
data_mean = data_mean.dropna()

In [8]:
t=t[data_mean.index]

In [9]:
t

region,1075,1076,1077,1125,1126,1127,1128,1129,1130,1131,...,1630,1684,1733,1734,1783,2068,2069,2118,2119,2168
2015-06-01 00:00:00,40,32,7,17,60,129,153,271,294,70,...,8,17,10,639,437,230,15,271,90,123
2015-06-01 01:00:00,17,20,5,13,38,90,111,163,246,28,...,6,7,22,547,490,212,21,237,124,119
2015-06-01 02:00:00,5,5,0,8,25,47,97,139,207,25,...,6,11,10,234,215,95,69,253,105,112
2015-06-01 03:00:00,5,6,0,6,27,26,68,77,208,29,...,13,0,2,84,10,17,6,78,3,94
2015-06-01 04:00:00,8,16,2,6,21,39,53,81,109,8,...,20,1,0,9,4,14,5,62,1,1
2015-06-01 05:00:00,14,28,17,14,41,49,55,101,60,17,...,6,0,1,13,11,44,10,71,1,56
2015-06-01 06:00:00,22,108,92,66,134,169,140,240,133,71,...,12,3,1,22,13,56,5,218,43,134
2015-06-01 07:00:00,51,183,183,106,186,333,280,367,198,140,...,8,4,8,27,250,92,7,156,55,88
2015-06-01 08:00:00,78,266,279,118,266,420,365,430,298,269,...,9,32,15,271,322,63,8,165,39,62
2015-06-01 09:00:00,75,229,174,70,260,349,335,488,345,232,...,4,6,9,373,323,74,9,125,33,138


In [10]:
t.to_csv('c6w4-00.csv', sep=';', index=True, index_label='index', header=True )

In [11]:
k

DatetimeIndex(['2015-06-01 00:00:00', '2015-06-01 01:00:00',
               '2015-06-01 02:00:00', '2015-06-01 03:00:00',
               '2015-06-01 04:00:00', '2015-06-01 05:00:00',
               '2015-06-01 06:00:00', '2015-06-01 07:00:00',
               '2015-06-01 08:00:00', '2015-06-01 09:00:00',
               ...
               '2016-06-30 14:00:00', '2016-06-30 15:00:00',
               '2016-06-30 16:00:00', '2016-06-30 17:00:00',
               '2016-06-30 18:00:00', '2016-06-30 19:00:00',
               '2016-06-30 20:00:00', '2016-06-30 21:00:00',
               '2016-06-30 22:00:00', '2016-06-30 23:00:00'],
              dtype='datetime64[ns]', length=9504, freq='H')